<a href="https://colab.research.google.com/github/AlekSSSandraJ/readnew/blob/main/NLP_CW_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖇 Deduplicating Training Data Makes Language Models

based on the code to deduplicate language model datasets as descrbed in the paper "Deduplicating Training Data Makes Language Models Better" by Katherine Lee, Daphne Ippolito, Andrew Nystrom, Chiyuan Zhang, Douglas Eck, Chris Callison-Burch and Nicholas Carlini.

In [32]:
import pandas as pd
import os
import glob
import re

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!ls "/content/drive/My Drive/Colab Notebooks/COP509cw/plagiarism/"
Data_path = "/content/drive/My Drive/Colab Notebooks/COP509cw/plagiarism/"

answers  metadata.csv  ReadMe.md  wikipedia


In [25]:
def styled_df(df):
    return df.style.set_table_styles(
        [{'selector': 'th',
          'props': [('background-color', '#add8e6'),  # Light blue header
                    ('color', 'black'),
                    ('font-weight', 'bold'),
                    ('text-align', 'center'),
                    ('font-family', 'Arial'),
                    ('border', '1.5px solid black')]},
         {'selector': 'td',
          'props': [('background-color', '#f4f4f4'),  # Light gray background
                    ('color', 'black'),
                    ('text-align', 'left'),
                    ('font-family', 'Arial'),
                    ('border', '1px solid lightgrey')]}]
    ).set_properties(**{'border': '1.5px solid lightgrey'})


In [26]:
def load_texts_from_directory(directory):
    text_data = []
    for file_path in glob.glob(os.path.join(directory, "*.txt")):
        try:
            with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                text_data.append({"Filename": os.path.basename(file_path), "Content": file.read()})
        except UnicodeDecodeError:
            print(f"Skipping file {file_path} due to encoding issues.")

    # Convert list of dictionaries into a DataFrame
    return pd.DataFrame(text_data)

In [27]:
Data_path = "/content/drive/My Drive/Colab Notebooks/COP509cw/plagiarism/"
WIKI_PATH = os.path.join(Data_path, "wikipedia/")
ANSWERS_PATH = os.path.join(Data_path, "answers/")

# Load Wikipedia articles and student answers
wiki_df = load_texts_from_directory(WIKI_PATH)
answers_df = load_texts_from_directory(ANSWERS_PATH)

# 🔍  Underdating Datasets  

In [28]:
print("Wikipedia Articles:")
display(styled_df(wiki_df))


Wikipedia Articles:


,Filename,Content
0,orig_taskc.txt,"Vector space model (or term vector model) is an algebraic model for representing text documents (and any objects, in general) as vectors of identifiers, such as, for example, index terms. It is used in information filtering, information retrieval, indexing and relevancy rankings. Its first use was in the SMART Information Retrieval System. A document is represented as a vector. Each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. One of the best known schemes is tf-idf weighting (see the example below). The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus). The vector space model has the following limitations: 1. Long documents are poorly represented because they have poor similarity values (a small scalar product and a large dimensionality) 2. Search keywords must precisely match document terms; word substrings might result in a ""false positive match"" 3. Semantic sensitivity; documents with similar context but different term vocabulary won't be associated, resulting in a ""false negative match"". 4. The order in which the terms appear in the document is lost in the vector space representation."
1,orig_taske.txt,"In mathematics and computer science, dynamic programming is a method of solving problems that exhibit the properties of overlapping subproblems and optimal substructure (described below). The method takes much less time than naive methods. The term was originally used in the 1940s by Richard Bellman to describe the process of solving problems where one needs to find the best decisions one after another. By 1953, he had refined this to the modern meaning. The field was founded as a systems analysis and engineering topic that is recognized by the IEEE. Bellman's contribution is remembered in the name of the Bellman equation, a central result of dynamic programming which restates an optimization problem in recursive form. The word ""programming"" in ""dynamic programming"" has no particular connection to computer programming at all, and instead comes from the term ""mathematical programming"", a synonym for optimization. Thus, the ""program"" is the optimal plan for action that is produced. For instance, a finalized schedule of events at an exhibition is sometimes called a program. Programming, in this sense, means finding an acceptable plan of action, an algorithm. Optimal substructure means that optimal solutions of subproblems can be used to find the optimal solutions of the overall problem. For example, the shortest path to a goal from a vertex in a graph can be found by first computing the shortest path to the goal from all adjacent vertices, and then using this to pick the best overall path, as shown in Figure 1. In general, we can solve a problem with optimal substructure using a three-step process: 1. Break the problem into smaller subproblems. 2. Solve these problems optimally using this three-step process recursively. 3. Use these optimal solutions to construct an optimal solution for the original problem. The subproblems are, themselves, solved by dividing them into sub-subproblems, and so on, until we reach some simple case that is solvable in constant time. Figure 2. The subproblem graph for the Fibonacci sequence. That it is not a tree but a DAG indicates overlapping subproblems. To say that a problem has overlapping subproblems is to say that the same subproblems are used to solve many different larger problems. For example, in the Fibonacci sequence, F3 = F1 + F2 and F4 = F2 + F3 — computing each number involves computing F2. Because both F3 and F4 are needed to compute F5, a nai

In [29]:
print("\nStudent Answers:")
display(styled_df(answers_df))


Student Answers:


,Filename,Content
0,g4pE_taskd.txt,"""Bayes' Theorem"" or ""Bayes' Rule"", or something called Bayesian reasoning The Bayesian Conspiracy is a multinational, interdisciplinary, and shadowy group of scientists that controls publication, grants, tenure, and the illicit traffic in grad students. The best way to be accepted into the Bayesian Conspiracy is to join the Campus Crusade for Bayes in high school or college, and gradually work your way up to the inner circles. . Bayes' Theorem Let and be sets. Conditional probability requires that (1) where denotes intersection (""and""), and also that (2) Therefore, (3) Now, let (4) so is an event in and for , then (5) (6) But this can be written (7) so This paper proposes a new measure called scaled inverse document frequency (SIDF) which evaluates the conditional specificity of query terms over a subset S of D and without making any assumption about term independence. S can be estimated from search results, OR searches, or computed from inverted index data. We have evaluated SIDF values from commercial search engines by submitting queries relevant to the financial investment domain. Results compare favorably across search engines and queries. Our approach has practical applications for `real-world scenarios like in Web Mining, Homeland Security, and keyword-driven marketing research scenarios."
1,g0pB_taskc.txt,"Vector space model is an algebraic model for representing text documents (and in general, any objects) as vectors of identifiers, such as, for example, index terms. Its first use was in the SMART Information Retrieval System. It is used in information filtering, information retrieval, indexing and relevancy rankings. A document is represented as a vector, and each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus). One of the best known schemes is tf-idf weighting, proposed by Salton, Wong and Yang. In the classic vector space model, the term specific weights in the document vectors are products of local and global parameters. Relevancy rankings of documents in a keyword search can be calculated, using the assumptions of document similarities theory, by comparing the deviation of angles between each document vector and the original query vector where the query is represented as same kind of vector as the documents. The vector space model has the following limitations: * Search keywords must precisely match document terms; word substrings might result in a ""false positive match""; * Semantic sensitivity; documents with similar context but different term vocabulary won't be associated, resulting in a ""false negative match""; * The order in which the terms appear in the document is lost in the vector space representation; * Long documents are poorly represented because they have poor similarity values (a small scalar product and a large dimensionality)."
2,g0pE_taskd.txt,"Bayes Theorem is an important theorem relating conditional probabilities, it allows us to calculate PROB(A|B) from PROB(B|A). Bayes Theorem is important because it can save us from gathering vast amounts of statistical evidence. The main theory is PROB(A|B) = PROB(B|A) * PROB(A) /PROB(B), it means Using PROB(WIN|RAIN) from earlier, we can find the probability that it rained on a day that Harry won a race."
3,g3pB_taskd.txt,"Bayes' theorem (often called Bayes' law) connects the conditional and marginal probabilities of two arbitrary events. One of its uses is calculating posterior probabilities given observations. Bayes' theorem plays a key role in the debate around the pr

# ⏳ PRE PROCESSING


Effective preprocessing ensures better accuracy in similarity detection and helps eliminate irrelevant variations that could lead to false positives or negatives. Before I implement a text similarity detection system using MinHash signatures, I will preprocess both wkipedia and the students answers.

MinHash is designed to capture similarity, and altering word forms by using lemmatization or stemming can reduce the effectiveness of exact matching since it changes the original substrings. As the main goal is to incorporate an automated classification framework to categorise detected similarities into four designated levels lemmatization and stemming won't be added to pre processing steps and I will focus instead on:

    - Converting words to lowercase
    - Removing punctuation and special characters
    - Tokenizes into words
    - Removes stopwords

In [30]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

In [33]:
wiki_df["Processed_Content"] = wiki_df["Content"].apply(preprocess_text)
answers_df["Processed_Content"] = answers_df["Content"].apply(preprocess_text)

Near Copy (Cut)

Answer provided by directly copying from Wikipedia article
Students selected relevant text within 200-300 word limit
No modifications to source text required
Light Revision (light)

Based on Wikipedia text with minor modifications allowed
Could substitute words/phrases with synonyms
Basic grammatical changes and paraphrasing permitted
Required to maintain original information order
Heavy Revision (heavy)

Based on Wikipedia but significantly rewritten
Freedom to split or combine sentences
No restrictions on restructuring while keeping meaning
Could modify text organization extensively
Non-Plagiarism (non)

Used provided learning materials (lecture notes/textbooks)
Written based on understanding, not copying
Could reference any materials except Wikipedia
Demonstrated acquired knowledge rather than text reuse

Varieties of plagiarism and their detection

Stein (2006) distinguish extrinsic and intrinsic plagiarism analysis

In extrinsic plagiarism analysis a key factor is the comparison of portions of text which it is suspected are plagiarised with their potential sources. This problem is made complex by the fact that there are a wide variety of ‘‘levels’’ of plagiarism. Martin (1994)

Intrinsic plagiarism analysis is generally carried out by identifying portions of a text written in a different style from the remainder and this is often carried out using stylometric features including surface characteristics (e.g. average sentence/word length), readability measures (e.g. Flesch-Kincaid Reading Ease Flesch 1974), Coleman-Liau Index (1975) and syntactic characteristics (e.g. part of speech and syntactic structure).


Near copy Participants were asked to answer the question by simply copying text from the relevant Wikipedia article (i.e. performing cut-and-paste actions). No instructions were given about which parts of the article to copy (selection had to be performed to produce a short answer of the required length, 200–300 words).

Light revision Participants were asked to base their answer on text found in the Wikipedia article and were, once again, given no instructions about which parts of the article to copy. They were instructed that they could alter the text in some basic ways including substituting words and phrases with synonyms and altering the grammatical structure (i.e. paraphrasing). Participants were also instructed not to radically alter the order of information found in sentences.

Heavy revision Participants were once again asked to base their answer on the relevant Wikipedia article but were instructed to rephrase the text to generate an answer with the same meaning as the source text, but expressed using different words and structure. This could include splitting source sentences into one or more individual sentences, or combining more than one source sentence into a single sentence. No constraints were placed on how the text could be altered.

Non-plagiarism Participants were provided with learning materials in the form of either lecture notes or sections from textbooks that could be used to answer the relevauestion. Participants were asked to read these materials and then attempt to answer the question using their own knowledge (including what they had learned


 Deduplication allows us to train models that emit memorized text ten times less frequently and require fewer train steps to achieve the same or better accuracy. We can also reduce train-test overlap, which affects over 4% of the validation set of standard datasets, thus allowing for more accurate evaluation [2]

 Deduplicating Training Data Makes Language Models Better

Katherine Lee, Daphne Ippolito, Andrew Nystrom, Chiyuan Zhang, Douglas Eck, Chris Callison-Burch, Nicholas Carlini